see https://kraina-ai.github.io/srai/latest/examples/regionalizers/voronoi_regionalizer/

Seems that there is a maximum of particles you can load in (also maybe there is a prolbem with points at the same place (too low precision?))
But it sort of works still. Maybe a workaround would be to start

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import numpy as np
import plotly.express as px
from shapely.geometry import Point
from shapely import geometry
from shapely.ops import unary_union
from srai.loaders.osm_loaders import OSMOnlineLoader
from srai.loaders.osm_loaders.filters.popular import get_popular_tags
from functional import seq

from srai.constants import WGS84_CRS
from srai.plotting.folium_wrapper import plot_regions
from srai.regionalizers import VoronoiRegionalizer, geocode_to_region_gdf, AdministrativeBoundaryRegionalizer
import srai as srai
import matplotlib.pyplot as plt
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions import  make_PDF, Haversine

from datetime import datetime, timedelta
import xarray as xr

# set plotscale
plt.style.use('../python_style_Meike.mplstyle')

In [ ]:
# importing GEOjson file with boundaries of all countries
# data downloaded from https://ec.europa.eu/eurostat/web/gisco/geodata/administrative-units/countries (make sure you download data in EPSG:4326 =WGS84 projection)

europe = gpd.read_file("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/input_data/europe.geojson")


In [ ]:
print(europe.geom_type)

In [ ]:
polygon = []
# list of countries in the domain
country_names = ['France','Germany','Denmark','Belgium','Netherlands','Bailiwick of Jersey','Jersey','Guernsey','Bailiwick of Guernsey','Isle of Man','United Kingdom','Ireland','Norway','Switzerland','Luxembourg','Italy','Liechtenstein','Austria']

for _, row in europe.iterrows():
    if(row['NAME_ENGL'] in country_names):
        print(row['NAME_ENGL'])
        polygon.append(row['geometry'])

land = unary_union(polygon)
landgdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(land,crs=WGS84_CRS))


In [ ]:
# define polygon simulation domain
domaingdf = gpd.GeoDataFrame(geometry=[
        geometry.Polygon(
            shell=[
                (9.977004051208496-0.5,61.28188705444336-0.5),
                (-15.996014595031738+0.5,61.28188705444336-0.5),
                (-15.996014595031738+0.5,46.00364303588867+0.5),
                (9.977004051208496-0.5,46.00364303588867+0.5),
                (9.977004051208496-0.5,61.28188705444336-0.5),
            ])],
              crs=WGS84_CRS,
            )



In [ ]:
domaingdf

In [ ]:
domaingdf_extended = gpd.GeoDataFrame(geometry=[
        geometry.Polygon(
            shell=[
                (9.977004051208496,61.28188705444336),
                (-15.996014595031738,61.28188705444336),
                (-15.996014595031738,46.00364303588867),
                (9.977004051208496,46.00364303588867),
                (9.977004051208496,61.28188705444336),
            ])],
              crs=WGS84_CRS,
            )


In [ ]:
seagdf = domaingdf.overlay(landgdf, how = 'difference')

# seagdf.explore()



In [ ]:
# # test reading in works :) 
# seagdf.to_file('/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/input_data/NWES_sea_mask.geojson', driver='GeoJSON')  
seagdf = gpd.GeoDataFrame.from_file('/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/input_data/NWES_sea_mask.geojson')
seagdf.explore()

In [ ]:
seagdf_extended = domaingdf_extended.overlay(landgdf, how = 'difference')


In [ ]:
# import data (test dataset is tracer september)
base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
input_file_tracer_base = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')

runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
runtime=timedelta(30)
nparticles = 88347 # 52511
chunck_time = 100
land_handling = 'anti_beaching'
coriolis = True
starttime = datetime(2023, 9, 1, 0, 0, 0, 0)
endtime = starttime + runtime

input_file_tracer = input_file_tracer_base.format(loc=loc,
                                                        y_s=starttime.year,
                                                        m_s=starttime.month,
                                                        d_s=starttime.day,
                                                        y_e=endtime.year,
                                                        m_e=endtime.month,
                                                        d_e=endtime.day,
    
                                                        land_handling = land_handling, 
                                                        cor_on = coriolis,
                                                        particle_type = 'tracer')
ds = xr.open_dataset(input_file_tracer,
                        engine='zarr',
                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                        drop_variables=['B','tau','z'],
                        decode_times=False) #,decode_cf=False)




In [ ]:

T=0
pointlist = [Point(np.round(lon,5), np.round(lat,5)) for lon, lat in zip(ds.lon[:,T].values,ds.lat[:,T].values) if np.isnan(lon)==False]
pointlist_unique = list(set(pointlist))

print(len(pointlist)-len(pointlist_unique))
# print(len(pointlist_unique))

In [ ]:


begin = 0#70255
end=len(pointlist_unique) #77255
seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pointlist_unique[begin:end]},
    index=list(range(len(pointlist_unique[begin:end]))),
    crs=WGS84_CRS,
)

In [ ]:
vr = VoronoiRegionalizer(seeds=seeds_gdf)
sea_results = vr.transform(gdf = seagdf)

In [ ]:
#  # plot without points
# folium_map = plot_regions(sea_results, tiles_style="CartoDB positron")
# seagdf.explore(
#     m=folium_map,
#     style_kwds=dict(color="#444", opacity=0, fillColor="#f2f2f2", fillOpacity=0)
    
# #     marker_kwds=dict(radius=0.1),
# )


# plot with points
# folium_map = plot_regions(sea_results, tiles_style="CartoDB positron")
# seeds_gdf.explore(
#     m=folium_map,
#     style_kwds=dict(color="#444", opacity=0, fillColor="#444", fillOpacity=1),
    
#   marker_kwds=dict(radius=1),
# )


In [ ]:


# regions = regionalizer.transform(sea_results)

# folium_map = plot_regions(sea_results, colormap=["rgba(0,0,0,0.1)"], tiles_style="CartoDB positron")
# seeds_gdf.explore(
#     m=folium_map,
#     style_kwds=dict(color="#444", opacity=0, fillColor="#444", fillOpacity=1),
    
#   marker_kwds=dict(radius=1),
# )

In [ ]:
#analysis area list
# see https://epsg.io/25832-1149 for epsg 25832 projection seems fine, but maybe find someone who has more experience with projections
arealist = sea_results.to_crs(epsg =3035).area
# print(arealist.type)
areas=[]
for _, area in arealist.items():
    areas.append(area/10**6)

    
areas = np.array(areas)
print(areas.size)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.sort(areas)[1000:-1000],'o')
mean_area=np.mean(areas)#np.sort(areas)[1000:-1000])
print(mean_area)

In [ ]:
bins, pdf = make_PDF(areas,nbins=1000,norm=True)
mean_area=np.mean(areas)
print(mean_area)
fig, ax = plt.subplots()
ax.plot((bins+bins[1])/mean_area,pdf[:],'--o',color='navy')
ax.axvline(1,color='grey',zorder=-5)
ax.set_xscale('log')
ax.set_xlabel('area / $\\langle$ area $\\rangle$')
ax.set_ylabel('PDF')

In [ ]:
binold = bins
pdfold = pdf
mean_area_old = 225

In [ ]:
fig, ax = plt.subplots()
ax.plot((bins[1:]+bins[1])/mean_area,pdf[1:],'--o',color='navy')
ax.plot((binold[1:]+binold[1])/mean_area_old,pdfold[1:],'--o',color='red')
